In [1]:
import gym
import torch
from torch import nn
from collections import namedtuple, deque
import itertools
from copy import deepcopy
import random
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import tqdm

/home/devjac/Code/python/learn-pytorch/.venv/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


In [2]:
RENDER = False

In [3]:
ALPHA_TARGET = 0.3      # How much we value entropy / exploration, increasing this will increase exploration.
GAMMA = 1 - 0.01        # How much we value future rewards.
TAU = 0.01              # How much q_target is updated when polyak averaging (step 15).
POLICY_LR = 0.001       # Policy learning rate.
Q_LR = 0.001            # Q learning rate.
ALPHA_LR = 0.001        # ALPHA learning rate.

In [4]:
env = gym.make("LunarLander-v2")

In [5]:
SARS = namedtuple('SARS', 'state, action, reward, next_state, t, failed, limit')

In [6]:
softmax = nn.Softmax(dim=0)
input = torch.tensor([1, 2, 3], dtype=float)
display(input)
output = softmax(input)
display(output)
sum(output)

tensor([1., 2., 3.], dtype=torch.float64)

tensor([0.0900, 0.2447, 0.6652], dtype=torch.float64)

tensor(1.0000, dtype=torch.float64)

In [7]:
softmax = nn.Softmax(dim=1)
input = torch.tensor([[1, 2, 3], [1, 2, 3], [3, 3, 3]], dtype=float)
display(input)
output = softmax(input)
display(output)
sum(output)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [3., 3., 3.]], dtype=torch.float64)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.3333, 0.3333, 0.3333]], dtype=torch.float64)

tensor([0.5134, 0.8228, 1.6638], dtype=torch.float64)

In [8]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 2000),
            nn.ReLU(),
            nn.Linear(2000, 1500),
            nn.ReLU(),
            nn.Linear(1500, output_size)
        )

    def forward(self, x):
        nn_out = self.linear_relu_stack(x)
        return nn.Softmax(dim=1)(nn_out)

    def __call__(self, x):
        raise RuntimeError("Use forward")

In [9]:
policy_network = PolicyNetwork(4, 2)
policy_network

PolicyNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=1500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1500, out_features=2, bias=True)
  )
)

In [10]:
torch.cuda.is_available()

True

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
mock_states = torch.rand(5, 4)
mock_states

tensor([[0.0328, 0.3989, 0.7453, 0.3104],
        [0.3251, 0.7370, 0.5895, 0.1613],
        [0.7599, 0.7150, 0.2383, 0.7809],
        [0.0727, 0.1167, 0.2260, 0.5418],
        [0.3726, 0.2841, 0.5720, 0.2513]])

In [13]:
policy_network.forward(mock_states)

tensor([[0.4977, 0.5023],
        [0.4966, 0.5034],
        [0.4977, 0.5023],
        [0.5095, 0.4905],
        [0.5037, 0.4963]], grad_fn=<SoftmaxBackward0>)

In [14]:
class QNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 2000),
            nn.ReLU(),
            nn.Linear(2000, 1500),
            nn.ReLU(),
            nn.Linear(1500, output_size)
        )

    def forward(self, x):
        nn_out = self.linear_relu_stack(x)
        return nn_out
    
    def __call__(self, x):
        raise RuntimeError("Use forward")

In [15]:
q_network = QNetwork(4, 2)
q_network

QNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=1500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1500, out_features=2, bias=True)
  )
)

In [16]:
q_network.forward(mock_states)

tensor([[-0.0431, -0.1055],
        [-0.0283, -0.1628],
        [-0.0539, -0.1224],
        [-0.0311, -0.0669],
        [-0.0473, -0.1118]], grad_fn=<AddmmBackward0>)

In [17]:
def run_episode(action_f, step_f, env, policy, fail_at_limit=False):
    episode_reward = 0
    s = env.reset()
    for t in itertools.count(start=1):
        a = action_f(policy, s)
        next_state, reward, failed, info = env.step(a)
        episode_reward += reward
        assert t <= env._max_episode_steps
        limit = t == env._max_episode_steps
        if limit and not fail_at_limit:
            failed = False
        assert fail_at_limit or not (limit and failed)
        step_f(s, a, reward, next_state, t, failed, limit)
        if failed or limit:
            break
        s = next_state
    return episode_reward

In [18]:
class Policy:
    def __init__(self, env_state_size, env_action_space_size):
        self.policy_network = PolicyNetwork(env_state_size, env_action_space_size)
        self.q1_network = QNetwork(env_state_size, env_action_space_size)
        self.q2_network = QNetwork(env_state_size, env_action_space_size)
        self.q1_target_network = deepcopy(self.q1_network)
        self.q2_target_network = deepcopy(self.q2_network)
        self.alpha = torch.tensor(ALPHA_TARGET, dtype=float, requires_grad=True)
        self.policy_network.to(device)
        self.q1_network.to(device)
        self.q2_network.to(device)
        self.q1_target_network.to(device)
        self.q2_target_network.to(device)
        self.alpha.to(device)
        self.reset_optimizers()

    def reset_optimizers(self):
        self.policy_optimizer = torch.optim.SGD(self.policy_network.parameters(), lr=POLICY_LR)
        self.q1_optimizer = torch.optim.SGD(self.q1_network.parameters(), lr=Q_LR)
        self.q2_optimizer = torch.optim.SGD(self.q2_network.parameters(), lr=Q_LR)
        self.alpha_optimizer = torch.optim.SGD([self.alpha], lr=ALPHA_LR)

    @property
    def alpha_dc(self):
        """Alpha, (D)etached and (C)lamped"""
        return self.alpha.detach().clamp(min=0)

In [19]:
t = torch.tensor(1, dtype=float, requires_grad=True)
display(t)
torch.optim.SGD([t], lr=ALPHA_LR)

tensor(1., dtype=torch.float64, requires_grad=True)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [20]:
replay_buffer = deque(maxlen=30_000)

In [21]:
env.action_space.n

4

In [22]:
env.observation_space.shape

(8,)

In [23]:
oss = env.observation_space.shape
if len(oss) != 1:
    raise RuntimeError(f'Unknown observation_space.shape: {oss}')
os_len = oss[0]
policy = Policy(os_len, env.action_space.n)

In [24]:
s = env.reset()
s

array([-0.00242882,  1.4054776 , -0.24603565, -0.2418847 ,  0.00282126,
        0.05573068,  0.        ,  0.        ], dtype=float32)

In [25]:
s = torch.tensor(s).reshape((1, -1))
s

tensor([[-0.0024,  1.4055, -0.2460, -0.2419,  0.0028,  0.0557,  0.0000,  0.0000]])

In [26]:
policy_output = policy.policy_network.forward(s.to(device))
policy_output

tensor([[0.2511, 0.2705, 0.2548, 0.2236]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [27]:
action_weights = policy_output.reshape((-1,)).tolist()

In [28]:
list(range(4))

[0, 1, 2, 3]

In [29]:
action = random.choices(range(len(action_weights)), weights=action_weights)[0]
action

3

In [30]:
def action(policy, s):
    tensor_s = torch.tensor(s).reshape((1, -1)).to(device)
    action_weights = policy.policy_network.forward(tensor_s).reshape((-1,)).tolist()
    action = random.choices(range(len(action_weights)), weights=action_weights)[0]
    return action

def step(initial_s, a, r, next_s, t, failed, limit):
    replay_buffer.append(SARS(initial_s, a, r, next_s, t, failed, limit))
    if RENDER:
        env.render()

In [31]:
run_episode(action, step, env, policy)

-532.0120235152353

In [32]:
len(replay_buffer)

92

In [33]:
replay_buffer

deque([SARS(state=array([-0.00621233,  1.4203664 , -0.6292554 ,  0.4198244 ,  0.00720531,
               0.14253572,  0.        ,  0.        ], dtype=float32), action=1, reward=-1.36716572510167, next_state=array([-0.01251898,  1.4292246 , -0.6401851 ,  0.39361894,  0.01661271,
               0.18816549,  0.        ,  0.        ], dtype=float32), t=1, failed=False, limit=False),
       SARS(state=array([-0.01251898,  1.4292246 , -0.6401851 ,  0.39361894,  0.01661271,
               0.18816549,  0.        ,  0.        ], dtype=float32), action=0, reward=-0.4159072851060728, next_state=array([-0.0188261 ,  1.4374851 , -0.64021426,  0.36699358,  0.02601443,
               0.18805172,  0.        ,  0.        ], dtype=float32), t=2, failed=False, limit=False),
       SARS(state=array([-0.0188261 ,  1.4374851 , -0.64021426,  0.36699358,  0.02601443,
               0.18805172,  0.        ,  0.        ], dtype=float32), action=1, reward=-1.6065209238964815, next_state=array([-0.02521667,  1.44

# Polyak Averaging

In [34]:
test_parameter_1 = next(policy.policy_network.named_parameters())[1]
test_parameter_1

Parameter containing:
tensor([[ 0.1246, -0.1029,  0.2727,  ..., -0.0717, -0.1187,  0.0831],
        [-0.1792, -0.1695,  0.0921,  ..., -0.3296,  0.0166, -0.2935],
        [ 0.1052, -0.1717, -0.0611,  ...,  0.1389,  0.3051, -0.1386],
        ...,
        [-0.1268,  0.3387, -0.1519,  ..., -0.2215,  0.3334,  0.2127],
        [ 0.1328, -0.0655,  0.2023,  ..., -0.2123, -0.2214, -0.2885],
        [-0.3128,  0.3353, -0.2610,  ...,  0.1156, -0.2113,  0.0482]],
       device='cuda:0', requires_grad=True)

In [35]:
test_parameter_2 = test_parameter_1 * 0 + 0.0128
test_parameter_2

tensor([[0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        ...,
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [36]:
test_parameter_1 * 0.9 + test_parameter_2 * 0.1

tensor([[ 0.1134, -0.0913,  0.2467,  ..., -0.0633, -0.1055,  0.0761],
        [-0.1600, -0.1513,  0.0841,  ..., -0.2954,  0.0162, -0.2629],
        [ 0.0960, -0.1532, -0.0537,  ...,  0.1263,  0.2759, -0.1234],
        ...,
        [-0.1129,  0.3061, -0.1355,  ..., -0.1981,  0.3013,  0.1927],
        [ 0.1208, -0.0576,  0.1834,  ..., -0.1898, -0.1980, -0.2584],
        [-0.2803,  0.3031, -0.2337,  ...,  0.1053, -0.1889,  0.0447]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [37]:
def polyak_update(network_to_update, target_network, tau=0.001):
    with torch.no_grad():
        for to_update, target in zip(network_to_update.parameters(), target_network.parameters()):
            to_update *= 1-tau
            to_update += target * tau

In [38]:
test_network_1 = QNetwork(5, 3)
test_network_2 = QNetwork(5, 3)
display(list(test_network_1.parameters())[0])
display(list(test_network_2.parameters())[0])
polyak_update(test_network_2, test_network_1, 0.1)
display(list(test_network_1.parameters())[0])
display(list(test_network_2.parameters())[0])

Parameter containing:
tensor([[ 0.1269, -0.4354,  0.0163, -0.1715, -0.3544],
        [ 0.3526, -0.1543,  0.0717,  0.2197,  0.1529],
        [-0.0542, -0.0805, -0.0672,  0.3208,  0.3937],
        ...,
        [ 0.3285, -0.0311,  0.0709, -0.3532, -0.2510],
        [-0.0835,  0.2199, -0.2628, -0.2567,  0.1461],
        [ 0.1652,  0.2135, -0.2146, -0.0699, -0.1571]], requires_grad=True)

Parameter containing:
tensor([[-0.4314, -0.0635, -0.1606, -0.0519, -0.0437],
        [ 0.1235,  0.4140,  0.0686,  0.3580, -0.3923],
        [-0.4411,  0.0470, -0.2552,  0.3637,  0.2513],
        ...,
        [ 0.4167, -0.2589, -0.3410,  0.2700, -0.4023],
        [-0.2164,  0.0037, -0.3848,  0.1103, -0.0099],
        [ 0.2880,  0.0448, -0.0536,  0.2964,  0.1785]], requires_grad=True)

Parameter containing:
tensor([[ 0.1269, -0.4354,  0.0163, -0.1715, -0.3544],
        [ 0.3526, -0.1543,  0.0717,  0.2197,  0.1529],
        [-0.0542, -0.0805, -0.0672,  0.3208,  0.3937],
        ...,
        [ 0.3285, -0.0311,  0.0709, -0.3532, -0.2510],
        [-0.0835,  0.2199, -0.2628, -0.2567,  0.1461],
        [ 0.1652,  0.2135, -0.2146, -0.0699, -0.1571]], requires_grad=True)

Parameter containing:
tensor([[-0.3755, -0.1007, -0.1429, -0.0639, -0.0748],
        [ 0.1464,  0.3572,  0.0689,  0.3442, -0.3378],
        [-0.4024,  0.0343, -0.2364,  0.3594,  0.2655],
        ...,
        [ 0.4079, -0.2362, -0.2998,  0.2077, -0.3872],
        [-0.2031,  0.0253, -0.3726,  0.0736,  0.0057],
        [ 0.2758,  0.0617, -0.0697,  0.2597,  0.1449]], requires_grad=True)

# Log Experiments

In [39]:
for p in [0.99, 0.9, 0.8, 0.6, 0.5]:
    logs = -np.log([p, 1-p])
    display(p, logs, sum(logs))

0.99

array([0.01005034, 4.60517019])

4.615220521841592

0.9

array([0.10536052, 2.30258509])

...

array([0.51082562, 0.91629073])

1.4271163556401456

0.5

array([0.69314718, 0.69314718])

1.3862943611198906

In [40]:
random.sample(replay_buffer, k=4)

[SARS(state=array([-0.95938766,  0.37276992, -1.8514162 , -1.855079  ,  2.5909123 ,
         0.6461136 ,  0.        ,  0.        ], dtype=float32), action=2, reward=-12.704562570869872, next_state=array([-0.9785686,  0.329327 , -1.8905493, -1.9417155,  2.6225498,
         0.6327504,  0.       ,  0.       ], dtype=float32), t=90, failed=False, limit=False),
 SARS(state=array([-0.23066053,  1.5267463 , -0.7958543 , -0.18790388,  0.59052384,
         0.55779785,  0.        ,  0.        ], dtype=float32), action=2, reward=-6.297212320205506, next_state=array([-0.23875633,  1.5228527 , -0.83265716, -0.18363586,  0.61818194,
         0.5531625 ,  0.        ,  0.        ], dtype=float32), t=33, failed=False, limit=False),
 SARS(state=array([-0.32390365,  1.4562562 , -0.95343935, -0.38829947,  0.9048682 ,
         0.56846344,  0.        ,  0.        ], dtype=float32), action=0, reward=-3.2010811070936143, next_state=array([-0.3332623 ,  1.4472615 , -0.95336026, -0.41500866,  0.93328977,
      

# Training

![Psudocode](sac_psudocode.png)

Source: https://spinningup.openai.com/en/latest/algorithms/sac.html#pseudocode

In [41]:
def q_min(q1, q2, states):
    
    def f(q):
        state_values = q.forward(states.to(device)).detach()
        return state_values
        
    return torch.minimum(*map(f, (q1, q2)))

In [42]:
def train(policy, replay_buffer):
    stats = {}
    # Step 11
    training_batch = random.sample(replay_buffer, k=min(len(replay_buffer), 100))
    # Prep
    states = torch.tensor(np.array([sars.state for sars in training_batch]), requires_grad=False).to(device)
    actions = torch.tensor(np.array([sars.action for sars in training_batch]), requires_grad=False).to(device)
    actions_hot = nn.functional.one_hot(actions, env.action_space.n).to(device)
    rewards = torch.tensor(np.array([sars.reward for sars in training_batch]), requires_grad=False).to(device)
    next_states = torch.tensor(np.array([sars.next_state for sars in training_batch]), requires_grad=False).to(device)
    fails = torch.tensor(np.array([sars.failed for sars in training_batch]), dtype=int, requires_grad=False).to(device)
    # Step 12
    next_action_probs = policy.policy_network.forward(next_states.to(device)).detach()
    assert not next_action_probs.requires_grad
    next_states_q_min = q_min(policy.q1_target_network, policy.q2_target_network, next_states)
    assert not next_states_q_min.requires_grad
    next_actions_q_min = torch.sum(next_states_q_min * next_action_probs, 1)
    assert not next_actions_q_min.requires_grad
    next_actions_entropy = torch.sum(next_action_probs * torch.log(next_action_probs), 1)
    assert not next_actions_entropy.requires_grad
    y = rewards + GAMMA * (1-fails) * (next_actions_q_min - policy.alpha_dc * next_actions_entropy)
    assert not y.requires_grad
    # Step 13
    for qi, q, opt in ((1, policy.q1_network, policy.q1_optimizer),
                       (2, policy.q2_network, policy.q2_optimizer)):
        assert not states.requires_grad
        assert not actions_hot.requires_grad
        q_state_action = torch.sum(q.forward(states.to(device)) * actions_hot, 1)
        assert q_state_action.requires_grad
        q_loss = torch.mean((q_state_action - y)**2)
        assert q_loss.requires_grad
        stats[f'train/q_loss_{qi}'] = q_loss.detach()
        assert q_loss.requires_grad
        opt.zero_grad()
        q_loss.backward()
        opt.step()
    # Step 14
    action_probs = policy.policy_network.forward(states)
    assert action_probs.requires_grad
    states_q_min = q_min(policy.q1_network, policy.q2_network, states)
    assert not states_q_min.requires_grad
    actions_q_min = torch.sum(states_q_min * action_probs, 1)
    assert actions_q_min.requires_grad
    actions_entropy = torch.sum(action_probs * torch.log(action_probs), 1)
    assert actions_entropy.requires_grad
    policy_loss = -1 * torch.mean(actions_q_min - policy.alpha_dc * actions_entropy)
    assert policy_loss.requires_grad
    stats['train/policy_loss'] = policy_loss.detach()
    policy.policy_optimizer.zero_grad()
    policy_loss.backward()
    policy.policy_optimizer.step()
    # Alpha Adjust
    assert policy.alpha.requires_grad
    assert action_probs.requires_grad
    expected_action_log_probs = torch.log(action_probs.detach()) * action_probs.detach()
    assert not expected_action_log_probs.requires_grad
    alpha_loss = torch.mean(-1 * policy.alpha * (expected_action_log_probs + ALPHA_TARGET))
    assert alpha_loss.requires_grad
    stats['train/alpha_loss'] = alpha_loss.detach()
    policy.alpha_optimizer.zero_grad()
    alpha_loss.backward()
    policy.alpha_optimizer.step()
    stats['train/alpha'] = policy.alpha.detach()
    # Step 15
    polyak_update(policy.q1_target_network, policy.q1_network, tau=TAU)
    polyak_update(policy.q2_target_network, policy.q2_network, tau=TAU)
    return stats

In [43]:
tb_writer = SummaryWriter()

oss = env.observation_space.shape
if len(oss) != 1:
    raise RuntimeError(f'Unknown observation_space.shape: {oss}')
os_len = oss[0]
policy = Policy(os_len, env.action_space.n)

replay_buffer = deque(maxlen=30_000)

def action(policy, s):
    tensor_s = torch.tensor(s).reshape((1, -1))
    action_weights = policy.policy_network.forward(tensor_s.to(device)).reshape((-1,)).tolist()
    action = random.choices(range(len(action_weights)), weights=action_weights)[0]
    return action

def step(initial_s, a, r, next_s, t, failed, limit):
    replay_buffer.append(SARS(initial_s, a, r, next_s, t, failed, limit))
    if RENDER:
        env.render()

for episode in tqdm.tqdm(range(1, 3000+1)):
    episode_reward = run_episode(action, step, env, policy, fail_at_limit=True)
    tb_writer.add_scalar('main/episode_reward', episode_reward, episode)
    tb_writer.add_scalar('main/replay_buffer_length', len(replay_buffer), episode)
    tb_writer.add_scalar('main/alpha_target', ALPHA_TARGET, episode)
    policy.reset_optimizers()
    for training_iteration in range(1, 100+1):
        stats = train(policy, replay_buffer)
        for stat, value in stats.items():
            tb_writer.add_scalar(stat, value, episode)
    if episode > 1000:
        ALPHA_TARGET -= 0.3 / 2000

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [46:54<00:00,  1.07it/s]


In [44]:
env = gym.make("LunarLander-v2")

def render_only(initial_s, a, r, next_s, t, failed, limit):
    env.render()

for episode in tqdm.tqdm(range(1, 10+1)):
    run_episode(action, render_only, env, policy)

env.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:00<00:00,  6.05s/it]
